# Check if GPU is avaliable

In [ ]:
!nvidia-smi

# Install Tensorflow Object Detection Requirements

In [ ]:
!apt-get update && apt-get install -y \
    protobuf-compiler \
    python3-pip \
    python3-pil \
    python3-lxml \
    python3-opencv

In [ ]:
!pip install -U pip
!pip install opencv-python-headless==4.1.2.30 tf_slim pycocotools

In [ ]:
import os
os.chdir('/')
!git clone https://github.com/tensorflow/models.git /tensorflow/models

In [ ]:
os.chdir('/tensorflow/models/research/')
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cp /tensorflow/models/research/object_detection/packages/tf2/setup.py /tensorflow/models/research
!cd /tensorflow/models/research/
!python -m pip install .

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
os.chdir('/tensorflow/models/research/')
!python object_detection/builders/model_builder_tf2_test.py

# Download Wrapper Code

In [ ]:
import os
os.chdir('/content/')
!git clone https://github.com/freds0/capybara_object_detection

In [ ]:
os.chdir('/content/capybara_object_detection')
!pip install -r requirements.txt

# Download Capybara Dataset

In [ ]:
os.chdir('/content/capybara_object_detection')
os.makedirs('data', exist_ok=True)

In [ ]:
os.chdir('/content/capybara_object_detection/data')
!git clone https://github.com/freds0/capybara_dataset

# Check Parameters

In [ ]:
!cat /content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Preprocess Data Augmentation

In [ ]:
os.chdir('/content/capybara_object_detection')
!python preprocess.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Create TFRecord Files

In [ ]:
os.chdir('/content/capybara_object_detection')
!python create_tfrecord.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Create Pipeline File Config

In [ ]:
os.chdir('/content/capybara_object_detection')
!python create_pipeline.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Train Model

In [ ]:
os.chdir('/content/capybara_object_detection')
!python train.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Check Training Logs

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/capybara_object_detection/checkpoints/train

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

# Evaluate Model

In [ ]:
os.chdir('/content/capybara_object_detection')
!python test.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

# Inference

First, export checkpoints to frozen graph.

In [ ]:
os.chdir('/content/capybara_object_detection')
!python export.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

Now, it's ready to inference at images folder.

In [ ]:
os.chdir('/content/capybara_object_detection')
!python inference.py --yaml=/content/capybara_object_detection/config/parameters_with_data_aug.yaml

In [ ]:
from IPython.display import Image, display
import os, random
inference_folder = '/content/capybara_object_detection/output_inference'
sample_filename = random.sample(os.listdir(inference_folder), 1)[0]
filepath = os.path.join(inference_folder, sample_filename)        
display(Image(filename=filepath))

In [ ]:
from google.colab import files
#from io import BytesIO
#from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))